In [42]:
import numpy as np
import pandas as pd
import time

Linear regression generally have the form of $Y_{i} = \theta_{0} + \theta_{1} x_{1} + \theta_{2} x_{2} + ...$ <br>
There are several ways to find the coefficients of the regression: <br>
1. Linear Algebra: $\hat{\theta} = (X^{T}X)^{-1}X^{T}Y$ (When X is invertible) <br>
2. Gradient Descent: In this case, we need to write out the loss function and try to minimize the loss. <br>
$\hspace{30mm}$ $F(x)$ = Loss Function = SE = $ \sum^{n}_{i=1} (Y_{i} - \hat{Y_{i}})^{2}$ <br>

In [307]:
import numpy as np
import pandas as pd

class Linear_Regression():
    def __init__(self, alpha = 1e-10 , num_iter = 10000, early_stop = 1e-50, intercept = True, init_weight = None):  
        """
        Initializes Linear Regression class.
        ---
        Parameters: 
            alpha: Learning Rate, defaults to 1e-10.
            num_iter: Number of Iterations to update coefficient with training data.
            early_stop: Constant control early_stop.
            intercept: Bool, If we are going to fit a intercept, default True.
            init_weight: Matrix (n x 1), input init_weight for testing.
        """
        self.model_name = 'Linear Regression'
        self.alpha = alpha
        self.num_iter = num_iter
        self.early_stop = early_stop
        self.intercept = intercept
        self.init_weight = init_weight  ### For testing correctness.
        self.coef = None
        self.loss = []
        

    def __str__(self):
        return self.model_name
    

    def fit(self, X_train, y_train):
        """
        Saves datasets to our model, and performs gradient descent.
        ---
        Parameters:
            X_train: Matrix or 2-D array. Input feature matrix.
            y_train: Matrix or 2-D array. Input target value.
        """
        self.X = np.mat(X_train)
        if isinstance(y_train, pd.core.series.Series):
            self.y = np.reshape(y_train.values, (-1, 1))
        elif y_train.ndim == 1:
            self.y = np.reshape(y_train, (-1, 1))
        else:
            self.y = y_train
        
        # adds column of all 1's to first column of X if intercept is True.
        if self.intercept:
            ones = np.ones((self.X.shape[0], 1))
            self.X = np.hstack((ones, self.X))
        
        if self.init_weight is not None:
            self.coef = self.init_weight
        else:
            # initializes coefficient with uniform from [-1, 1]
            self.coef = np.random.rand(self.X.shape[1]) * 2 - 1
            self.coef = np.reshape(self.coef, (-1, 1))

        # Call gradient_descent function to train.
        self.gradient_descent()
        

    def gradient(self):
        """
        Helper function to calculate the gradient of the cost function.
        Gradient: -2X^T(y - Xw)
        ---
        Returns the gradient of the cost function.
        """
        y_pred = np.dot(self.X, self.coef)
        y_pred = np.reshape(y_pred, (-1, 1))
        error = np.subtract(self.y, y_pred)

        # Return gradient of cost function
        return -2 * np.dot(self.X.T, error)
        

    def gradient_descent(self):
        """
        Performs gradient descent to find the model's optimal coefficients.
        Cost function: SE = \sum^{n}_{i=1}(Y_i - \hat{Y}_i)^2
        """
        for i in range(self.num_iter):
            if i == 0:
                pre_error = np.sum(np.power(np.dot(self.X, self.coef) - self.y, 2))
                
            # Calculate the gradient and temporary coefficient to compare
            temp_coef = self.coef - self.alpha * self.gradient()  

            # Calculate error for early stopping
            y_pred = np.dot(self.X, temp_coef)
            current_error = np.sum(np.power(y_pred - self.y, 2))

            ### This is the early stop, don't modify the following three lines.
            if (abs(pre_error - current_error) < self.early_stop) | (abs(abs(pre_error - current_error) / pre_error) < self.early_stop):
                self.coef = temp_coef
                return self
            
            # Update learning rate and coefficients based on error comparison
            if current_error < pre_error:
                self.coef = temp_coef
                # Increase learning rate if error decreases
                self.alpha *= 1.3 
            else:
                # Decrease learning rate if error increases
                self.alpha *= 0.9 

            # Update previous error for next iteration
            pre_error = min(pre_error, current_error)
            # Add loss to loss list we create
            self.loss.append(min(pre_error, current_error))

            if i % 1000 == 0:
                print('Iteration: ' + str(i))
                print('Coef:      ' + str(self.coef))
                print('Loss:      ' + str(current_error))
                print('Alpha:     ' + str(self.alpha)) 
    
        return self


    def ind_predict(self, x: list):
        """
        Predict the value based on its feature vector x.
        ---
        Parameters:
            x: Matrix, array or list. Input feature point.
        ---
        Returns a prediction of given data point.
        """
        x = np.matrix(x)
        return np.array(np.dot(x, self.coef)).flatten()


    def predict(self, X):
        """
            X is a matrix or 2-D numpy array, represnting testing instances. 
            Each testing instance is a feature vector. 
            ---
            Parameters:
                X: Matrix, array or list. Input feature point.
            ---
            Returns a prediction of the given data matrix.
        """
        X = np.matrix(X)

        # Adds ones if intercept is True.
        if self.intercept:
            ones = np.ones((X.shape[0], 1))
            X = np.hstack((ones, X))
            
        # Calls ind_predict for each value in the X matrix.
        predictions = np.array([self.ind_predict(x) for x in X]).flatten()
        return predictions



In [256]:
clf = Linear_Regression(alpha = 1, num_iter = 10000000)
clf.fit(X,y)

Iteration: 0
Coef:      [[-0.91057242]
 [-0.41744811]
 [-0.67575189]]
Loss:      4.028959561318494e+16
Alpha:     0.9
Iteration: 1000
Coef:      [[-0.65679721]
 [-0.16288564]
 [ 0.61493265]]
Loss:      926.8218867288629
Alpha:     5.764558317550444e-06
Iteration: 2000
Coef:      [[-0.52295548]
 [-0.02775224]
 [ 0.59254355]]
Loss:      903.504796077026
Alpha:     6.877290656446413e-06
Iteration: 3000
Coef:      [[-0.40313087]
 [ 0.09327846]
 [ 0.56954518]]
Loss:      885.0575735271298
Alpha:     5.680255604228824e-06
Iteration: 4000
Coef:      [[-0.29505537]
 [ 0.20249108]
 [ 0.546423  ]]
Loss:      870.5465946806013
Alpha:     6.776714995536758e-06
Iteration: 5000
Coef:      [[-0.19793905]
 [ 0.30067193]
 [ 0.52828915]]
Loss:      857.7870357836855
Alpha:     5.5971857602931585e-06
Iteration: 6000
Coef:      [[-0.10998647]
 [ 0.3896327 ]
 [ 0.51352466]]
Loss:      848.2103702246509
Alpha:     6.677610184715114e-06
Iteration: 7000
Coef:      [[-0.03130352]
 [ 0.46926641]
 [ 0.49838677]]

Iteration: 68000
Coef:      [[0.66834145]
 [1.20150822]
 [0.36171415]]
Loss:      805.5176819231256
Alpha:     6.109124832580431e-06
Iteration: 69000
Coef:      [[0.6682276 ]
 [1.20185742]
 [0.3616854 ]]
Loss:      805.517594571224
Alpha:     7.288368824764271e-06
Iteration: 70000
Coef:      [[0.66810283]
 [1.20219435]
 [0.3616647 ]]
Loss:      805.5175161892075
Alpha:     6.019783070204819e-06
Iteration: 71000
Coef:      [[0.66796774]
 [1.20252212]
 [0.36165275]]
Loss:      805.5174353167045
Alpha:     7.1817814274704295e-06
Iteration: 72000
Coef:      [[0.66782293]
 [1.20283871]
 [0.36163654]]
Loss:      805.5173602835667
Alpha:     5.931747869852923e-06
Iteration: 73000
Coef:      [[0.66766841]
 [1.20314727]
 [0.3616162 ]]
Loss:      805.5172855961283
Alpha:     7.0767527977877e-06
Iteration: 74000
Coef:      [[0.66750737]
 [1.20344737]
 [0.36160372]]
Loss:      805.5172129631572
Alpha:     5.845000123950884e-06
Iteration: 75000
Coef:      [[0.66733914]
 [1.20374192]
 [0.3615965 ]]


KeyboardInterrupt: 

In [167]:
def min_max_normaliz(lst):
    """
    Helper function for normalize for faster training.
    """
    maximum = np.max(lst)
    minimum = np.min(lst)

    return (lst - minimum) / (maximum - minimum)

### We generate some easy data for testing. We should fit a line with, $Y = 30 * X + 20$

In [223]:
X = np.array(np.mat(np.arange(1, 1000, 5)).T)
y = np.array((30 * X)).flatten() + 20

#### Do NOT modify the following line, just run it when you are done.  You can also try different initialization, you will notice different coef at the end.

In [198]:
clf = Linear_Regression(alpha = 1, num_iter = 10000000, init_weight= np.mat([15,25]).T)
clf.fit(X,y)

Iteration: 0
Coef:      [[15]
 [25]]
Loss:      2.931949540902693e+25
Alpha:     0.9
Iteration: 1000
Coef:      [[15.00927684]
 [30.00748189]]
Loss:      1250.9887008778671
Alpha:     1.60528649192378e-08
Iteration: 2000
Coef:      [[15.0114471 ]
 [30.00751175]]
Loss:      1249.8963903412282
Alpha:     1.9151548451188908e-08
Iteration: 3000
Coef:      [[15.01360825]
 [30.0075084 ]]
Loss:      1248.824872042921
Alpha:     1.581810277533585e-08
Iteration: 4000
Coef:      [[15.0157756]
 [30.0074717]]
Loss:      1247.7244230806502
Alpha:     1.8871470185031235e-08
Iteration: 5000
Coef:      [[15.01793566]
 [30.00746476]]
Loss:      1246.6650689874657
Alpha:     1.5586773866839906e-08
Iteration: 6000
Coef:      [[15.02010111]
 [30.00749851]]
Loss:      1245.5988198093428
Alpha:     1.8595487871499764e-08
Iteration: 7000
Coef:      [[15.02225851]
 [30.00749529]]
Loss:      1244.4920314864953
Alpha:     1.5358827985036065e-08
Iteration: 8000
Coef:      [[15.02442272]
 [30.00745634]]
Loss:    

Iteration: 69000
Coef:      [[15.15445143]
 [30.0072619 ]]
Loss:      1179.2637360631454
Alpha:     2.029629917514577e-08
Iteration: 70000
Coef:      [[15.15655031]
 [30.00726026]]
Loss:      1178.2541315728604
Alpha:     1.676360254261788e-08
Iteration: 71000
Coef:      [[15.15865651]
 [30.00729126]]
Loss:      1177.2200912681312
Alpha:     1.9999479714469813e-08
Iteration: 72000
Coef:      [[15.1607536]
 [30.0072864]]
Loss:      1176.2115816806256
Alpha:     1.6518446348242352e-08
Iteration: 73000
Coef:      [[15.16285697]
 [30.00725145]]
Loss:      1175.1713204803884
Alpha:     1.9707001034912428e-08
Iteration: 74000
Coef:      [[15.16495375]
 [30.00724227]]
Loss:      1174.1917864000516
Alpha:     1.6276875395135137e-08
Iteration: 75000
Coef:      [[15.16705455]
 [30.00727556]]
Loss:      1173.1326985783758
Alpha:     1.9418799655525682e-08
Iteration: 76000
Coef:      [[15.16914797]
 [30.00727123]]
Loss:      1172.1214534470412
Alpha:     1.603883725160054e-08
Iteration: 77000
Coef

Iteration: 141000
Coef:      [[15.3035568 ]
 [30.00707177]]
Loss:      1107.824626671242
Alpha:     1.724979750322579e-08
Iteration: 142000
Coef:      [[15.30559149]
 [30.00706897]]
Loss:      1106.8452321954228
Alpha:     1.4247363363604124e-08
Iteration: 143000
Coef:      [[15.30763202]
 [30.0070336 ]]
Loss:      1105.8964431862082
Alpha:     1.6997531040877536e-08
Iteration: 144000
Coef:      [[15.30967236]
 [30.00706296]]
Loss:      1104.921578797435
Alpha:     2.0278563416417953e-08
Iteration: 145000
Coef:      [[15.31170371]
 [30.00705732]]
Loss:      1103.9709411640222
Alpha:     1.6748953802591952e-08
Iteration: 146000
Coef:      [[15.31374279]
 [30.00702119]]
Loss:      1103.0091723669068
Alpha:     1.9982003329054106e-08
Iteration: 147000
Coef:      [[15.31577245]
 [30.00702125]]
Loss:      1102.0600520184048
Alpha:     1.650401183599643e-08
Iteration: 148000
Coef:      [[15.31780824]
 [30.00704716]]
Loss:      1101.0845854507731
Alpha:     1.9689780229651923e-08
Iteration: 1

Iteration: 213000
Coef:      [[15.44807408]
 [30.00682115]]
Loss:      1040.6805978798589
Alpha:     1.4660579809873497e-08
Iteration: 214000
Coef:      [[15.4500527 ]
 [30.00685055]]
Loss:      1039.788466358443
Alpha:     1.7490510632456384e-08
Iteration: 215000
Coef:      [[15.45202361]
 [30.00684675]]
Loss:      1038.8674348102413
Alpha:     1.4446178881171554e-08
Iteration: 216000
Coef:      [[15.4540018 ]
 [30.00681004]]
Loss:      1038.0037384670206
Alpha:     1.7234723905621428e-08
Iteration: 217000
Coef:      [[15.45597757]
 [30.00684248]]
Loss:      1037.0647429669702
Alpha:     2.056154160531273e-08
Iteration: 218000
Coef:      [[15.457946  ]
 [30.00683895]]
Loss:      1036.1817379104073
Alpha:     1.6982677884303916e-08
Iteration: 219000
Coef:      [[15.45992132]
 [30.00680105]]
Loss:      1035.2728298236807
Alpha:     2.0260843155940805e-08
Iteration: 220000
Coef:      [[15.46188733]
 [30.00680228]]
Loss:      1034.3805868300021
Alpha:     1.6734317863250135e-08
Iteration:

Iteration: 285000
Coef:      [[15.58815063]
 [30.00661119]]
Loss:      977.6409658698716
Alpha:     1.7997788116895945e-08
Iteration: 286000
Coef:      [[15.59006219]
 [30.00660729]]
Loss:      976.7740527195891
Alpha:     1.4865161576222587e-08
Iteration: 287000
Coef:      [[15.59197891]
 [30.00663704]]
Loss:      975.9346340780656
Alpha:     1.7734582804631006e-08
Iteration: 288000
Coef:      [[15.59388951]
 [30.00663905]]
Loss:      975.0857746560125
Alpha:     1.4647768779445198e-08
Iteration: 289000
Coef:      [[15.59580436]
 [30.00659964]]
Loss:      974.2478764811975
Alpha:     1.7475226689609295e-08
Iteration: 290000
Coef:      [[15.59771189]
 [30.00659891]]
Loss:      973.3770265891926
Alpha:     1.443355520328021e-08
Iteration: 291000
Coef:      [[15.59962651]
 [30.00662836]]
Loss:      972.5623611715245
Alpha:     1.721966347996002e-08
Iteration: 292000
Coef:      [[15.60153241]
 [30.0066231 ]]
Loss:      971.6897364762326
Alpha:     1.4222474353805874e-08
Iteration: 293000


...

Iteration: 9712000
Coef:      [[19.92632013]
 [30.00011045]]
Loss:      0.27266178206302705
Alpha:     1.588945600456426e-08
Iteration: 9713000
Coef:      [[19.92635217]
 [30.00011092]]
Loss:      0.2724241920945766
Alpha:     1.8956596723726476e-08
Iteration: 9714000
Coef:      [[19.92638408]
 [30.00011088]]
Loss:      0.27219160723451236
Alpha:     1.565708360400928e-08
Iteration: 9715000
Coef:      [[19.92641607]
 [30.00011032]]
Loss:      0.27194976238460766
Alpha:     1.867936949292768e-08
Iteration: 9716000
Coef:      [[19.92644796]
 [30.00011021]]
Loss:      0.27171852479473535
Alpha:     1.5428109490502333e-08
Iteration: 9717000
Coef:      [[19.92647993]
 [30.0001107 ]]
Loss:      0.2714862003933215
Alpha:     1.8406196520317572e-08
Iteration: 9718000
Coef:      [[19.92651177]
 [30.00011062]]
Loss:      0.27124356625465484
Alpha:     1.5202483966425028e-08
Iteration: 9719000
Coef:      [[19.92654374]
 [30.00011006]]
Loss:      0.27101831348122285
Alpha:     1.8137018515149647e-

Iteration: 9784000
Coef:      [[19.92858748]
 [30.00010706]]
Loss:      0.25613667978197596
Alpha:     1.9506395120217923e-08
Iteration: 9785000
Coef:      [[19.92861843]
 [30.00010697]]
Loss:      0.25591915963999157
Alpha:     1.611118618289905e-08
Iteration: 9786000
Coef:      [[19.92864947]
 [30.00010748]]
Loss:      0.25569471041987046
Alpha:     1.922112746480189e-08
Iteration: 9787000
Coef:      [[19.92868037]
 [30.00010737]]
Loss:      0.2554730322256715
Alpha:     1.5875571130499985e-08
Iteration: 9788000
Coef:      [[19.92871138]
 [30.00010684]]
Loss:      0.2552496955280562
Alpha:     1.8940031653272208e-08
Iteration: 9789000
Coef:      [[19.92874226]
 [30.00010681]]
Loss:      0.2550300670118777
Alpha:     1.564340178670906e-08
Iteration: 9790000
Coef:      [[19.92877323]
 [30.00010721]]
Loss:      0.2548056360952872
Alpha:     1.8663046675272173e-08
Iteration: 9791000
Coef:      [[19.9288041]
 [30.0001072]]
Loss:      0.25458761873808705
Alpha:     1.5414627760400124e-08
I

Iteration: 9856000
Coef:      [[19.93078497]
 [30.00010423]]
Loss:      0.24062031002606518
Alpha:     1.6578459104195533e-08
Iteration: 9857000
Coef:      [[19.93081505]
 [30.00010371]]
Loss:      0.24040676756009208
Alpha:     1.9778598049470846e-08
Iteration: 9858000
Coef:      [[19.93084504]
 [30.00010359]]
Loss:      0.2402059441713649
Alpha:     1.6336010505676093e-08
Iteration: 9859000
Coef:      [[19.93087509]
 [30.00010405]]
Loss:      0.23998939009549028
Alpha:     1.948934961283169e-08
Iteration: 9860000
Coef:      [[19.93090504]
 [30.00010403]]
Loss:      0.23978426118914226
Alpha:     1.6097107551691725e-08
Iteration: 9861000
Coef:      [[19.93093508]
 [30.00010352]]
Loss:      0.23957378534628077
Alpha:     1.9204331236275093e-08
Iteration: 9862000
Coef:      [[19.93096499]
 [30.0001035 ]]
Loss:      0.23936692322500114
Alpha:     1.5861698389622036e-08
Iteration: 9863000
Coef:      [[19.93099502]
 [30.00010392]]
Loss:      0.2391588817647649
Alpha:     1.8923481058072448

Iteration: 9927000
Coef:      [[19.93288575]
 [30.00010056]]
Loss:      0.2262380013950245
Alpha:     1.7059284347493237e-08
Iteration: 9928000
Coef:      [[19.93291482]
 [30.00010052]]
Loss:      0.22603688349961767
Alpha:     1.4090010203095355e-08
Iteration: 9929000
Coef:      [[19.93294398]
 [30.00010096]]
Loss:      0.22584259989434424
Alpha:     1.6809804009435302e-08
Iteration: 9930000
Coef:      [[19.93297315]
 [30.0001004 ]]
Loss:      0.22564523814011458
Alpha:     2.005459944759667e-08
Iteration: 9931000
Coef:      [[19.93300217]
 [30.00010041]]
Loss:      0.225451283029536
Alpha:     1.6563972150282445e-08
Iteration: 9932000
Coef:      [[19.93303129]
 [30.0001008 ]]
Loss:      0.2252508920307095
Alpha:     1.9761314679731322e-08
Iteration: 9933000
Coef:      [[19.93306032]
 [30.00010086]]
Loss:      0.22506430438008873
Alpha:     1.6321735413531974e-08
Iteration: 9934000
Coef:      [[19.93308941]
 [30.00010027]]
Loss:      0.22486139217597373
Alpha:     1.9472319000525567e-

Iteration: 9999000
Coef:      [[19.93495096]
 [30.00009797]]
Loss:      0.2125244361730363
Alpha:     1.4498662933810355e-08


####  As the number of iteration increase, you should notice the coeficient converges to [20, 30]. 
#### It maybe very slow update. Feel free to stop.

In [199]:
clf.coef

matrix([[19.93497924],
        [30.00009741]])

In [200]:
np.array(clf.predict(X))

[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.93497924]
 [30.00009741]]
[[19.934

array([[[   49.93507666]],

       [[  199.93556373]],

       [[  349.9360508 ]],

       [[  499.93653787]],

       [[  649.93702495]],

       [[  799.93751202]],

       [[  949.93799909]],

       [[ 1099.93848616]],

       [[ 1249.93897323]],

       [[ 1399.93946031]],

       [[ 1549.93994738]],

       [[ 1699.94043445]],

       [[ 1849.94092152]],

       [[ 1999.9414086 ]],

       [[ 2149.94189567]],

       [[ 2299.94238274]],

       [[ 2449.94286981]],

       [[ 2599.94335688]],

       [[ 2749.94384396]],

       [[ 2899.94433103]],

       [[ 3049.9448181 ]],

       [[ 3199.94530517]],

       [[ 3349.94579225]],

       [[ 3499.94627932]],

       [[ 3649.94676639]],

       [[ 3799.94725346]],

       [[ 3949.94774053]],

       [[ 4099.94822761]],

       [[ 4249.94871468]],

       [[ 4399.94920175]],

       [[ 4549.94968882]],

       [[ 4699.9501759 ]],

       [[ 4849.95066297]],

       [[ 4999.95115004]],

       [[ 5149.95163711]],

       [[ 5299.95212

#### Please try to normalize the X and fit again with normalized X. You should find something interesting. Also think about what you should do for predicting.

##### You can also try this with the wine dataset we use in HW1. Try fit this function to that dataset with same features. If you look closely to the updates of coefficients. What do you find? This could be mentioned in your report. 

In [298]:
from sklearn.linear_model import LinearRegression

In [299]:
url_Wine = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
wine = pd.read_csv(url_Wine, delimiter=';')
X = wine[['density','alcohol']]
y = wine.quality

In [300]:
lr = LinearRegression()
lr.fit(X,y)
## Squared Error with sklearn.
sum((lr.predict(X) - y)**2)

800.6676988774321

#### You will notice different coefficients, but the loss is very close to each other like 805. In your report, briefly discuss this problem.

In [ ]:
clf = Linear_Regression(alpha = 1, num_iter = 5000000)
clf.fit(X,y)

Iteration: 0
Coef:      [[-0.3686047 ]
 [-0.36555888]
 [-0.04524747]]
Loss:      9580751193045918.0
Alpha:     0.9
Iteration: 1000
Coef:      [[-0.19908892]
 [-0.19516322]
 [ 0.57463693]]
Loss:      891.8399911492044
Alpha:     5.76455831755044e-06
Iteration: 2000
Coef:      [[-0.08633873]
 [-0.08124868]
 [ 0.55605721]]
Loss:      875.6865930229742
Alpha:     6.877290656446416e-06
Iteration: 3000
Coef:      [[0.01457449]
 [0.02075665]
 [0.53666511]]
Loss:      862.1130697064242
Alpha:     5.680255604228824e-06
Iteration: 4000
Coef:      [[0.10555781]
 [0.11277389]
 [0.51718889]]
Loss:      851.6345294727317
Alpha:     6.776714995536759e-06
Iteration: 5000
Coef:      [[0.18735499]
 [0.19554431]
 [0.50171254]]
Loss:      842.7297391627302
Alpha:     5.597185760293166e-06
Iteration: 6000
Coef:      [[0.261434  ]
 [0.2705489 ]
 [0.48945772]]
Loss:      835.9842353284581
Alpha:     6.677610184715129e-06
Iteration: 7000
Coef:      [[0.32764718]
 [0.33763882]
 [0.47652924]]
Loss:      829.989

Iteration: 62000
Coef:      [[0.9153237 ]
 [0.95462419]
 [0.36161885]]
Loss:      805.5891155346039
Alpha:     6.385184846638417e-06
Iteration: 63000
Coef:      [[0.91527832]
 [0.95504574]
 [0.36159041]]
Loss:      805.5889994788729
Alpha:     7.617716686424246e-06
Iteration: 64000
Coef:      [[0.91521381]
 [0.95544652]
 [0.36155857]]
Loss:      805.5888978207618
Alpha:     6.291805895818738e-06
Iteration: 65000
Coef:      [[0.91513164]
 [0.9558312 ]
 [0.36152338]]
Loss:      805.5887990003303
Alpha:     7.5063128024483024e-06
Iteration: 66000
Coef:      [[0.91503538]
 [0.95619986]
 [0.36149829]]
Loss:      805.588708291011
Alpha:     6.199792548135935e-06
Iteration: 67000
Coef:      [[0.91492557]
 [0.95655684]
 [0.36148091]]
Loss:      805.5886210442412
Alpha:     7.3965381238990905e-06
Iteration: 68000
Coef:      [[0.91480338]
 [0.95689958]
 [0.36146001]]
Loss:      805.5885396384301
Alpha:     6.10912483258038e-06
Iteration: 69000
Coef:      [[0.91466922]
 [0.95723171]
 [0.36143546]

In [ ]:
sum((clf.predict(X) - y)**2)